In [ ]:
import os
import json
import xml.etree.ElementTree as ET
from PIL import Image

# Directory paths
kaggle_annotations_dir = '../data/kaggle-dataset/train'
kaggle_images_dir = "../data/kaggle-dataset/train"
output_json_file = "combined_annotations.json"

# Initialize COCO format dictionary
coco_format = {
    "info": {
        "description": "TrashCAN Segmentation Dataset",
        "url": "N/A",
        "version": "0.5",
        "year": 2020,
        "contributor": "YourNameOrOrganization",
        "date_created": "2024/12/07"
    },
    "licenses": [
        {
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }
    ],
    "images": [],
    "annotations": []
}


annotation_id = 1


def parse_xml(file_path, image_id):
    tree = ET.parse(file_path)
    root = tree.getroot()
    annotations = []
    
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        width = xmax - xmin
        height = ymax - ymin
        
        annotation = {
            "id": len(coco_format["annotations"]) + 1,
            "image_id": image_id,
            "category_id": 1,
            "bbox": [xmin, ymin, width, height],
            "area": width * height,
            "iscrowd": 0
        }
        annotations.append(annotation)
    
    return annotations


image_id = 6009
for file_name in os.listdir(kaggle_annotations_dir):
    file_path = os.path.join(kaggle_annotations_dir, file_name)
    if file_name.endswith(".xml"):

        image_name = file_name.replace(".xml", ".jpg")
        image_path = os.path.join(kaggle_images_dir, image_name)
        
        if not os.path.exists(image_path):
            continue
        

        with Image.open(image_path) as img:
            width, height = img.size
        

        coco_format["images"].append({
            "id": image_id,
            "file_name": image_name,
            "width": width,
            "height": height
        })
        

        annotations = parse_xml(file_path, image_id)
        coco_format["annotations"].extend(annotations)
        image_id += 1


with open(output_json_file, "w") as json_file:
    json.dump(coco_format, json_file, indent=4)

print(f"Annotations converted and saved to {output_json_file}")

Annotations converted and saved to combined_annotations.json


In [10]:
#combining the combined_annotations.json with instances_train_trashcan.json
with open('combined_annotations.json') as f:
    data = json.load(f)
with open('data/material_version/instances_train_trashcan.json') as f:
    data2 = json.load(f)

data['categories'] = data2['categories']
data['info'] = data2['info']
data['licenses'] = data2['licenses']

with open('combined_annotations.json', 'w') as f:
    json.dump(data, f)

print(f"Annotations combined and saved to instanes_train_trashcan.json")

Annotations combined and saved to instanes_train_trashcan.json


In [ ]:
# combining the imaages from kaggle-dataset and material_version
import shutil
import os

src = 'data/kaggle-dataset/train'
dst = 'data/material_version/train'

for file in os.listdir(src):
    if file.endswith(".jpg"):
        shutil.copy(os.path.join(src, file), dst)

print(f"Images combined and saved to data/material_version/train")

Images combined and saved to data/material_version/train
